# Análise Exploratória de Dados: IBGE

<h1>
    <p align="center">
        Work in Progress 🛠️
    </p>
</h1>

Objetivos:

- Realizar uma análise focada nas variáveis referentes a população brasileira e alguns índices econômicos;

Algumas possibilidades: 

- Comparativos entre estados ou regiões do Brasil no quesito populacional e/ou econômico;
- Procurar entender quais os estados com os municípios mais ricos, mais populosos, com mais imigrantes;
- Rankeamento desses estados com base em alguns KPI's (GDP, IDH, População total);
- Um dado interessante para a análise também pode ser a distribuição de carros/motos por cidadão.
- Divisão etária da população desses estados, buscando quais os de população mais jovem e velha na média;

## Puxando dados e selecionando variáveis necessárias

In [230]:
# Importando pacotes:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Definindo parâmetros de tamanho das figuras do Seaborn:
sns.set(rc={'figure.figsize':(12,8)})
sns.set(style="darkgrid")

print('Pacotes prontos!')

Pacotes prontos!


In [231]:
# Criando dataframe:
data = pd.read_csv('data/BRAZIL_CITIES.csv', sep=';')

In [232]:
# Visualizando dados:
data.head()

,CITY,STATE,CAPITAL,IBGE_RES_POP,IBGE_RES_POP_BRAS,IBGE_RES_POP_ESTR,IBGE_DU,IBGE_DU_URBAN,IBGE_DU_RURAL,IBGE_POP,...,Pu_Bank,Pr_Assets,Pu_Assets,Cars,Motorcycles,Wheeled_tractor,UBER,MAC,WAL-MART,POST_OFFICES
0,Abadia De Goiás,GO,0,6876.0,6876.0,0.0,2137.0,1546.0,591.0,5300.0,...,NaN,NaN,NaN,2158.0,1246.0,0.0,NaN,NaN,NaN,1.0
1,Abadia Dos Dourados,MG,0,6704.0,6704.0,0.0,2328.0,1481.0,847.0,4154.0,...,NaN,NaN,NaN,2227.0,1142.0,0.0,NaN,NaN,NaN,1.0
2,Abadiânia,GO,0,15757.0,15609.0,148.0,4655.0,3233.0,1422.0,10656.0,...,1.0,33724584.0,67091904.0,2838.0,1426.0,0.0,NaN,NaN,NaN,3.0
3,Abaeté,MG,0,22690.0,22690.0,0.0,7694.0,6667.0,1027.0,18464.0,...,2.0,44974716.0,371922572.0,6928.0,2953.0,0.0,NaN,NaN,NaN,4.0
4,Abaetetuba,PA,0,141100.0,141040.0,60.0,31061.0,19057.0,12004.0,82956.0,...,4.0,76181384.0,800078483.0,5277.0,25661.0,0.0,NaN,NaN,NaN,2.0


In [233]:
# Checando shape:
data.shape

(5573, 81)

Dados: 5573 Registros em 81 Variáveis

In [234]:
# Checando colunas:
data.columns

Index(['CITY', 'STATE', 'CAPITAL', 'IBGE_RES_POP', 'IBGE_RES_POP_BRAS',
       'IBGE_RES_POP_ESTR', 'IBGE_DU', 'IBGE_DU_URBAN', 'IBGE_DU_RURAL',
       'IBGE_POP', 'IBGE_1', 'IBGE_1-4', 'IBGE_5-9', 'IBGE_10-14',
       'IBGE_15-59', 'IBGE_60+', 'IBGE_PLANTED_AREA', 'IBGE_CROP_PRODUCTION_$',
       'IDHM Ranking 2010', 'IDHM', 'IDHM_Renda', 'IDHM_Longevidade',
       'IDHM_Educacao', 'LONG', 'LAT', 'ALT', 'PAY_TV', 'FIXED_PHONES', 'AREA',
       'REGIAO_TUR', 'CATEGORIA_TUR', 'ESTIMATED_POP', 'RURAL_URBAN',
       'GVA_AGROPEC', 'GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC',
       ' GVA_TOTAL ', 'TAXES', 'GDP', 'POP_GDP', 'GDP_CAPITA', 'GVA_MAIN',
       'MUN_EXPENDIT', 'COMP_TOT', 'COMP_A', 'COMP_B', 'COMP_C', 'COMP_D',
       'COMP_E', 'COMP_F', 'COMP_G', 'COMP_H', 'COMP_I', 'COMP_J', 'COMP_K',
       'COMP_L', 'COMP_M', 'COMP_N', 'COMP_O', 'COMP_P', 'COMP_Q', 'COMP_R',
       'COMP_S', 'COMP_T', 'COMP_U', 'HOTELS', 'BEDS', 'Pr_Agencies',
       'Pu_Agencies', 'Pr_Bank', 'Pu_Bank', 'P

Antes de nos aprofundarmos mais no Dataset, iremos realizar um corte de variáveis, preservando apenas aquelas que interessam para a análise proposta no início.

In [235]:
# Selecionando as variáveis que serão relevantes para a análise:
data_pop = data.loc[
    :,
    ['CITY', 'STATE', 'CAPITAL', 'IBGE_RES_POP', 'IBGE_RES_POP_BRAS', 'IBGE_RES_POP_ESTR', 'IBGE_DU', 
    'IBGE_DU_URBAN', 'IBGE_DU_RURAL', 'IBGE_POP', 'IBGE_1', 'IBGE_1-4', 'IBGE_5-9', 'IBGE_10-14',
    'IBGE_15-59', 'IBGE_60+', 'Cars', 'Motorcycles', 'IDHM', 'IDHM_Renda', 
    'IDHM_Longevidade', 'IDHM_Educacao', 'TAXES', 'LAT', 'LONG']
].copy()


In [236]:
# Visualizando novos dados:
data_pop.head()

,CITY,STATE,CAPITAL,IBGE_RES_POP,IBGE_RES_POP_BRAS,IBGE_RES_POP_ESTR,IBGE_DU,IBGE_DU_URBAN,IBGE_DU_RURAL,IBGE_POP,...,IBGE_60+,Cars,Motorcycles,IDHM,IDHM_Renda,IDHM_Longevidade,IDHM_Educacao,TAXES,LAT,LONG
0,Abadia De Goiás,GO,0,6876.0,6876.0,0.0,2137.0,1546.0,591.0,5300.0,...,416.0,2158.0,1246.0,0.708,0.687,0.830,0.622,20554.20,-16.758812,-49.440548
1,Abadia Dos Dourados,MG,0,6704.0,6704.0,0.0,2328.0,1481.0,847.0,4154.0,...,589.0,2227.0,1142.0,0.690,0.693,0.839,0.563,12873.50,-18.487565,-47.396832
2,Abadiânia,GO,0,15757.0,15609.0,148.0,4655.0,3233.0,1422.0,10656.0,...,990.0,2838.0,1426.0,0.690,0.671,0.841,0.579,26822.58,-16.182672,-48.718812
3,Abaeté,MG,0,22690.0,22690.0,0.0,7694.0,6667.0,1027.0,18464.0,...,2681.0,6928.0,2953.0,0.698,0.720,0.848,0.556,26994.09,-19.155848,-45.446191
4,Abaetetuba,PA,0,141100.0,141040.0,60.0,31061.0,19057.0,12004.0,82956.0,...,5996.0,5277.0,25661.0,0.628,0.579,0.798,0.537,95180.48,-1.723470,-48.884404


Com nossas variáveis selecionadas, podemos dar início propriamente as análises iniciais do Dataset e limpeza.

## Análise inicial e conhecendo os dados 

In [237]:
# Visualizando o novo shape:
data_pop.shape

(5573, 25)

In [238]:
data_pop.columns

Index(['CITY', 'STATE', 'CAPITAL', 'IBGE_RES_POP', 'IBGE_RES_POP_BRAS',
       'IBGE_RES_POP_ESTR', 'IBGE_DU', 'IBGE_DU_URBAN', 'IBGE_DU_RURAL',
       'IBGE_POP', 'IBGE_1', 'IBGE_1-4', 'IBGE_5-9', 'IBGE_10-14',
       'IBGE_15-59', 'IBGE_60+', 'Cars', 'Motorcycles', 'IDHM', 'IDHM_Renda',
       'IDHM_Longevidade', 'IDHM_Educacao', 'TAXES', 'LAT', 'LONG'],
      dtype='object')

Agora temos os mesmos 5573 registros em apenas 23 variáveis

Para termos uma visão melhor e mais apropiada do conteúdo de cada variável, realizaremos um dicionário de variáveis, baseado no arquivo CSV presente na pasta `data` chamado `Data_Dictionary.csv`:


| <p align='center'>**Variável**</p>| <p align='center'>**Conteúdo**</p>                                                         |
|-----------------------------------|--------------------------------------------------------------------------------------------|
| *`CITY`*                          | Nome da cidade.                                                                            |
| *`STATE`*                         | Nome do estado.                                                                            |
| *`CAPITAL`*                       | Indica se a cidade nomeada no registro é a capital do estado.                              |
| *`IBGE_RES_POP`*                  | Número de habitantes da cidade de acordo com o senso de 2010.                              |
| *`IBGE_RES_POP_BRAS`*             | Quantos cidadãos da população total da cidade são brasileiros.                             |
| *`IBGE_RES_POP_ESTR`*             | Quantos cidadãos da população total da cidade são estrangeiros.                            |  
| *`IBGE_DU`*                       | Número de unidades domésticas totais.                                                      |
| *`IBGE_DU_URBAN`*                 | Das unidades domésticas totais, quantas estão em regiões urbanas.                          |
| *`IBGE_DU_RURAL`*                 | Das unidades domésticas totais, quantas estão em regiões rurais.                           |
| *`IBGE_POP`*                      | Moradores em domicílios particulares permanentes, em áreas com ordenamento urbano regular. |
| *`IBGE_1`*                        | Número de pessoas com até 1 ano de idade na população referente a tabela `IBGE_POP`.       |
| *`IBGE_1-4`*                      | Número de pessoas entre 1 e 4 anos de idade na população referente a tabela `IBGE_POP`.    |
| *`IBGE_5-9`*                      | Número de pessoas entre 5 e 9 anos de idade na população referente a tabela `IBGE_POP`.    |
| *`IBGE_10-14`*                    | Número de pessoas entre 10 e 14 anos de idade na população referente a tabela `IBGE_POP`.  |
| *`IBGE_15-59`*                    | Número de pessoas entre 15 e 59 anos de idade na população referente a tabela `IBGE_POP`.  |
| *`IBGE_60+`*                      | Número de pessoas a partir dos 60 anos de idade na população referente a tabela `IBGE_POP`.|
| *`Cars`*                          | Número total de carros referente ao ano de 2019.                                           |
| *`Motorcycles`*                   | Número total de Motocicletas referente ao ano de 2019.                                     |
| *`IDHM`*                          | Índice de Desenvolvimento Humano.                                                          |
| *`IDHM_Renda`*                    | Índice de Desenvolvimento Humano - Renda.                                                  |
| *`IDHM_Longevidade`*              | Índice de Desenvolvimento Humano - Longevidade.                                            |
| *`IDHM_Educacao`*                 | Índice de Desenvolvimento Humano - Educação                                                |
| *`TAXES`*                         | Montante total pago em Impostos.                                                           |
| *`LONG`*                          | Coordenada longitudinal da cidade.                                                         |
| *`LAT`*                           | Coordenada latitudinal da cidade.                                                          |

Os nomes das variáveis serão alterados mais adiante, para facilitar a compreensão das mesmas sem a necessidade de referenciar a tabela a todo momento. Este processo também sera realizado para padronizar a nomenclatura da melhor maneira possível.

In [239]:
# Checando informações específicas sobre as variáveis:
data_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5573 entries, 0 to 5572
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CITY               5573 non-null   object 
 1   STATE              5573 non-null   object 
 2   CAPITAL            5573 non-null   int64  
 3   IBGE_RES_POP       5565 non-null   float64
 4   IBGE_RES_POP_BRAS  5565 non-null   float64
 5   IBGE_RES_POP_ESTR  5565 non-null   float64
 6   IBGE_DU            5563 non-null   float64
 7   IBGE_DU_URBAN      5563 non-null   float64
 8   IBGE_DU_RURAL      5492 non-null   float64
 9   IBGE_POP           5565 non-null   float64
 10  IBGE_1             5565 non-null   float64
 11  IBGE_1-4           5565 non-null   float64
 12  IBGE_5-9           5565 non-null   float64
 13  IBGE_10-14         5565 non-null   float64
 14  IBGE_15-59         5565 non-null   float64
 15  IBGE_60+           5565 non-null   float64
 16  Cars               5562 

Algumas informações relevantes:

- No total, temos 22 variáveis do tipo float64, 1 do tipo int64 e 2 do tipo object. Visando a melhor performance possível do Dataset, iremos avaliar a possibilidade de alterar os Data Types para opções melhores (int8 ou 32 por exemplo, categorical para as variáveis object, etc.);

- Temos poucos dados faltantes espalhados em diversas colunas.

Dado o fato de que praticamente todas as colunas estão em float por mais que muitas não precisem estar, a avaliação estatística utilizando o método .describe() fica prejudicada, assim como a procura por outliers utilizando boxplots. Sendo assim, antes de continuar essas análises estatísticas, iremos realizar a retirada de valores nulos e as transformações de Data Types nos dados.

## Retirada de Nulls, Alterando Dtypes e Renomenado Colunas

In [240]:
# Criando cópia dos dados para a limpeza:
data_pop_clean = data_pop.copy()

- Renomenado Colunas:

In [241]:
# Renomeando e padronizando colunas:
data_pop_clean = data_pop_clean.rename(columns={
    'CITY': 'City', 'STATE': 'State', 'CAPITAL': 'Capital',
    'IBGE_RES_POP': 'Total_Pop', 'IBGE_RES_POP_BRAS': 'Total_Pop_Brazilians',
    'IBGE_RES_POP_ESTR': 'Total_Pop_Foreigners', 'IBGE_DU': 'Total_Uni_Dom',
    'IBGE_DU_URBAN': 'Total_Uni_Dom_Urb', 'IBGE_DU_RURAL': 'Total_Uni_Dom_Rur',
    'IBGE_POP': 'Total_Pop_Urb', 'IBGE_1': 'Pop_Urb_1', 'IBGE_1-4': 'Pop_Urb_1-4',
    'IBGE_5-9': 'Pop_Urb_5-9', 'IBGE_10-14': 'Pop_Urb_10-14', 'IBGE_15-59': 'Pop_Urb_15-19',
    'IBGE_60+': 'Pop_Urb_60+', 'Cars': 'Total_Cars', 'Motorcycles': 'Total_Motorcycles',
    'TAXES': 'Taxes', 'LAT': 'City_Latitude', 'LONG': 'City_Longitude'
})

In [242]:
data_pop_clean.head()

,City,State,Capital,Total_Pop,Total_Pop_Brazilians,Total_Pop_Foreigners,Total_Uni_Dom,Total_Uni_Dom_Urb,Total_Uni_Dom_Rur,Total_Pop_Urb,...,Pop_Urb_60+,Total_Cars,Total_Motorcycles,IDHM,IDHM_Renda,IDHM_Longevidade,IDHM_Educacao,Taxes,City_Latitude,City_Longitude
0,Abadia De Goiás,GO,0,6876.0,6876.0,0.0,2137.0,1546.0,591.0,5300.0,...,416.0,2158.0,1246.0,0.708,0.687,0.830,0.622,20554.20,-16.758812,-49.440548
1,Abadia Dos Dourados,MG,0,6704.0,6704.0,0.0,2328.0,1481.0,847.0,4154.0,...,589.0,2227.0,1142.0,0.690,0.693,0.839,0.563,12873.50,-18.487565,-47.396832
2,Abadiânia,GO,0,15757.0,15609.0,148.0,4655.0,3233.0,1422.0,10656.0,...,990.0,2838.0,1426.0,0.690,0.671,0.841,0.579,26822.58,-16.182672,-48.718812
3,Abaeté,MG,0,22690.0,22690.0,0.0,7694.0,6667.0,1027.0,18464.0,...,2681.0,6928.0,2953.0,0.698,0.720,0.848,0.556,26994.09,-19.155848,-45.446191
4,Abaetetuba,PA,0,141100.0,141040.0,60.0,31061.0,19057.0,12004.0,82956.0,...,5996.0,5277.0,25661.0,0.628,0.579,0.798,0.537,95180.48,-1.723470,-48.884404


- Remoção de valores nulos:

In [243]:
# Checando quantos valores nulos estão presentes em cada coluna:
data_pop_clean.isnull().sum()

City                     0
State                    0
Capital                  0
Total_Pop                8
Total_Pop_Brazilians     8
Total_Pop_Foreigners     8
Total_Uni_Dom           10
Total_Uni_Dom_Urb       10
Total_Uni_Dom_Rur       81
Total_Pop_Urb            8
Pop_Urb_1                8
Pop_Urb_1-4              8
Pop_Urb_5-9              8
Pop_Urb_10-14            8
Pop_Urb_15-19            8
Pop_Urb_60+              8
Total_Cars              11
Total_Motorcycles       11
IDHM                     8
IDHM_Renda               8
IDHM_Longevidade         8
IDHM_Educacao            8
Taxes                    3
City_Latitude            9
City_Longitude           9
dtype: int64

In [244]:
# Pesquisando quais são as latitudes e longitudes faltantes:
data_pop_clean.loc[(data_pop_clean['City_Latitude'].isnull()), ['City', 'State', 'City_Latitude', 'City_Longitude']]

,City,State,City_Latitude,City_Longitude
471,Balneário Rincão,SC,NaN,NaN
2701,Lagoa Dos Patos,RS,NaN,NaN
3116,Mojuí Dos Campos,PA,NaN,NaN
3580,Paraíso Das Águas,MS,NaN,NaN
3760,Pescaria Brava,SC,NaN,NaN
3805,Pinhal Da Serra,RS,NaN,NaN
3820,Pinto Bandeira,RS,NaN,NaN
4489,Santa Terezinha,BA,NaN,NaN
4605,São Caetano,PE,NaN,NaN


 - Checando caso a caso:

In [245]:
data_pop_clean.loc[(data_pop_clean['City_Latitude'].isnull()), :].T


,471,2701,3116,3580,3760,3805,3820,4489,4605
City,Balneário Rincão,Lagoa Dos Patos,Mojuí Dos Campos,Paraíso Das Águas,Pescaria Brava,Pinhal Da Serra,Pinto Bandeira,Santa Terezinha,São Caetano
State,SC,RS,PA,MS,SC,RS,RS,BA,PE
Capital,0,0,0,0,0,0,0,0,0
Total_Pop,NaN,NaN,NaN,NaN,NaN,2130.0,NaN,9648.0,NaN
Total_Pop_Brazilians,NaN,NaN,NaN,NaN,NaN,2130.0,NaN,9648.0,NaN
Total_Pop_Foreigners,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
Total_Uni_Dom,NaN,NaN,NaN,NaN,NaN,745.0,NaN,2891.0,NaN
Total_Uni_Dom_Urb,NaN,NaN,NaN,NaN,NaN,180.0,NaN,734.0,NaN
Total_Uni_Dom_Rur,NaN,NaN,NaN,NaN,NaN,565.0,NaN,2157.0,NaN
Total_Pop_Urb,NaN,NaN,NaN,NaN,NaN,478.0,NaN,2332.0,NaN


Com a tabela, temos registros 100% faltantes como Lagoa dos Patos no Rio Grande do Sul e uma característica semelhante a todos os registros, os dados que mais aparecem completos na tabela são o número total de carros e motos.

Devido a quantidade de dados faltantes, os registros referentes a:
- Balneário Rincão;
- Lagoa Dos Patos;
- Mojuí Dos Campos;
- Paraíso Das Águas;
- Pescaria Brava;
- Pinto Bandeira e
- São Caetano

Serão removidos. Já os registros de Pinhal Da Serra e Santa Terezinha podem ser tratados.

In [246]:
# Excluíndo registros desnecessários:
data_pop_clean = data_pop_clean.drop(data_pop_clean[data_pop_clean['Total_Pop'].isnull()].index)

In [247]:
# Checando dados que sobraram:
data_pop_clean.loc[(data_pop_clean['City_Latitude'].isnull()), :].T

,3805,4489
City,Pinhal Da Serra,Santa Terezinha
State,RS,BA
Capital,0,0
Total_Pop,2130.0,9648.0
Total_Pop_Brazilians,2130.0,9648.0
Total_Pop_Foreigners,0.0,0.0
Total_Uni_Dom,745.0,2891.0
Total_Uni_Dom_Urb,180.0,734.0
Total_Uni_Dom_Rur,565.0,2157.0
Total_Pop_Urb,478.0,2332.0


Com esses dois registros, basta inserirmos as latitudes e longitudes para poder realizar o preenchimento dos nulos.

In [248]:
# Preenchendo dados utilziando o método .at[]:

# Pinhal da Serra:
data_pop_clean.at[3805, 'City_Latitude'] = -27.874407
data_pop_clean.at[3805, 'City_Longitude'] = -51.171822

# Santa Terezinha:
data_pop_clean.at[4489, 'City_Latitude'] = -12.770203
data_pop_clean.at[4489, 'City_Longitude'] = -39.524779

In [249]:
# Checando:
data_pop_clean[['City_Latitude', 'City_Longitude']].isnull().sum()

City_Latitude     0
City_Longitude    0
dtype: int64

In [250]:
data_pop_clean.isnull().sum()

City                     0
State                    0
Capital                  0
Total_Pop                0
Total_Pop_Brazilians     0
Total_Pop_Foreigners     0
Total_Uni_Dom            2
Total_Uni_Dom_Urb        2
Total_Uni_Dom_Rur       73
Total_Pop_Urb            0
Pop_Urb_1                0
Pop_Urb_1-4              0
Pop_Urb_5-9              0
Pop_Urb_10-14            0
Pop_Urb_15-19            0
Pop_Urb_60+              0
Total_Cars               9
Total_Motorcycles        9
IDHM                     1
IDHM_Renda               1
IDHM_Longevidade         1
IDHM_Educacao            1
Taxes                    0
City_Latitude            0
City_Longitude           0
dtype: int64